In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches


%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [2]:
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [3]:
# This is a callback function to be used with training of Keras models.
# It create an exponential moving average of a model (trainable) weights.
# This functionlity is already available in TensorFlow:
# https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#ExponentialMovingAverage
# and can often be used to get better validation/test performance. For an
# intuitive explantion on why to use this, see 'Model Ensembles" section here:
# http://cs231n.github.io/neural-networks-3/

import numpy as np
import scipy.sparse as sp

from keras import backend as K
from keras.callbacks import Callback
from keras.models import load_model

import sys
import warnings


class ExponentialMovingAverage(Callback):
    """create a copy of trainable weights which gets updated at every
       batch using exponential weight decay. The moving average weights along
       with the other states of original model(except original model trainable
       weights) will be saved at every epoch if save_mv_ave_model is True.
       If both save_mv_ave_model and save_best_only are True, the latest
       best moving average model according to the quantity monitored
       will not be overwritten. Of course, save_best_only can be True
       only if there is a validation set.
       This is equivalent to save_best_only mode of ModelCheckpoint
       callback with similar code. custom_objects is a dictionary
       holding name and Class implementation for custom layers.
       At end of every batch, the update is as follows:
       mv_weight -= (1 - decay) * (mv_weight - weight)
       where weight and mv_weight is the ordinal model weight and the moving
       averaged weight respectively. At the end of the training, the moving
       averaged weights are transferred to the original model.
       """
    def __init__(self, decay=0.999, filepath='temp_weight.hdf5',
                 save_mv_ave_model=True, verbose=0,
                 save_best_only=False, monitor='val_loss', mode='auto',
                 save_weights_only=False, custom_objects={}):
        self.decay = decay
        self.filepath = filepath
        self.verbose = verbose
        self.save_mv_ave_model = save_mv_ave_model
        self.save_weights_only = save_weights_only
        self.save_best_only = save_best_only
        self.monitor = monitor
        self.custom_objects = custom_objects  # dictionary of custom layers
        self.sym_trainable_weights = None  # trainable weights of model
        self.mv_trainable_weights_vals = None  # moving averaged values
        super(ExponentialMovingAverage, self).__init__()

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_train_begin(self, logs={}):
        self.sym_trainable_weights = self.model.trainable_weights
        # Initialize moving averaged weights using original model values
        self.mv_trainable_weights_vals = {x.name: K.get_value(x) for x in
                                          self.sym_trainable_weights}
        if self.verbose:
            print('Created a copy of model weights to initialize moving'
                  ' averaged weights.')

    def on_batch_end(self, batch, logs={}):
        for weight in self.sym_trainable_weights:
            old_val = self.mv_trainable_weights_vals[weight.name]
            self.mv_trainable_weights_vals[weight.name] -= \
                (1.0 - self.decay) * (old_val - K.get_value(weight))

    def on_epoch_end(self, epoch, logs={}):
        """After each epoch, we can optionally save the moving averaged model,
        but the weights will NOT be transferred to the original model. This
        happens only at the end of training. We also need to transfer state of
        original model to model2 as model2 only gets updated trainable weight
        at end of each batch and non-trainable weights are not transferred
        (for example mean and var for batch normalization layers)."""
        if self.save_mv_ave_model:
            filepath = self.filepath.format(epoch=epoch, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best moving averaged model only '
                                  'with %s available, skipping.'
                                  % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('saving moving average model to %s'
                                  % (filepath))
                        self.best = current
                        model2 = self._make_mv_model(filepath)
                        if self.save_weights_only:
                            model2.save_weights(filepath, overwrite=True)
                        else:
                            model2.save(filepath, overwrite=True)
            else:
                if self.verbose > 0:
                    print('Epoch %05d: saving moving average model to %s' % (epoch, filepath))
                model2 = self._make_mv_model(filepath)
                if self.save_weights_only:
                    model2.save_weights(filepath, overwrite=True)
                else:
                    model2.save(filepath, overwrite=True)

    def on_train_end(self, logs={}):
        for weight in self.sym_trainable_weights:
            K.set_value(weight, self.mv_trainable_weights_vals[weight.name])

    def _make_mv_model(self, filepath):
        """ Create a model with moving averaged weights. Other variables are
        the same as original mode. We first save original model to save its
        state. Then copy moving averaged weights over."""
        self.model.save(filepath, overwrite=True)
        model2 = load_model(filepath, custom_objects=self.custom_objects)

        for w2, w in zip(model2.trainable_weights, self.model.trainable_weights):
            K.set_value(w2, self.mv_trainable_weights_vals[w.name])

        return model2


def batch_generator(X, y=None, batch_size=128, shuffle=False):
    index = np.arange(X.shape[0])

    while True:
        if shuffle:
            np.random.shuffle(index)

        batch_start = 0
        while batch_start < X.shape[0]:
            batch_index = index[batch_start:batch_start + batch_size]
            batch_start += batch_size

            X_batch = X[batch_index, :]

            if sp.issparse(X_batch):
                X_batch = X_batch.toarray()

            if y is None:
                yield X_batch
            else:
                yield X_batch, y[batch_index]

In [4]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

# np.log(train['loss'] + 200) provides
# a better score, but let's keep it simple now
train_y = np.array(train['loss'])

test = pd.read_csv('test.csv')

print (train_x.shape)
print (train_y.shape)
print (test.shape)

(188318, 1153)
(188318,)
(125546, 131)


<img src="http://cdn.rawgit.com/dnkirill/allstate_capstone/master/images/mlp3.svg"></td>

In [5]:
from keras import regularizers
from keras.initializers import he_normal
from keras.regularizers import l2
def hyper_model(seed = None):
    model = Sequential()
    model.add(Dense(437, input_dim=train_x.shape[1], init=he_normal(seed = seed)
                    ,W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))
    
    model.add(Dense(182, init=he_normal(seed = seed),W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(73, init=he_normal(seed = seed),W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.233))
    
    model.add(Dense(1, init=he_normal(seed = seed),W_regularizer=l2(0.002)))
    model.compile(loss='mae', optimizer='adadelta')
    return model

In [8]:
nnmodel={}
def cross_validate_mlp(mlp_func, nfolds=10,nbags=10):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nbags,))
    stack_train = np.zeros((nbags,len(train_y)))
    stack_test = np.zeros((nbags,len(test)))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        for bag in range(nbags):
            nnmodel['nn%d',k*10+bag*1] = mlp_func(seed = k*10+bag*1)
            #early_stopping = EarlyStopping(monitor='val_loss', patience=5)
            fit = nnmodel['nn%d',k*10+bag*1].fit(xtr, ytr, validation_split=0.2, batch_size=128,
                          nb_epoch=30, verbose=1, callbacks=[ExponentialMovingAverage(save_mv_ave_model=False)])
            pred = nnmodel['nn%d',k*10+bag*1].predict(xte, batch_size=256)
            score = mean_absolute_error(yte, pred)
            val_scores[bag] += score
            #stack_train[bag][test_index] = pred[:,0]
    for bag in range(nbags):
        avg_score = val_scores[bag] / float(nfolds)
    print (avg_score)
    return val_scores

In [ ]:
cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(437, input_dim=1153, kernel_initializer=<keras.ini..., kernel_regularizer=<keras.reg...)`
  import sys
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(182, kernel_initializer=<keras.ini..., kernel_regularizer=<keras.reg...)`
  if sys.path[0] == '':
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(73, kernel_initializer=<keras.ini..., kernel_regularizer=<keras.reg...)`
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer=<keras.ini..., kernel_regularizer=<keras.reg...)`
/home/coder.chenshicheng/anaconda3/lib/

Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 20s 145us/step - loss: 2136.8445 - val_loss: 1211.0096
Epoch 2/30
135588/135588 [==============================] - 17s 125us/step - loss: 1368.0154 - val_loss: 1174.2123
Epoch 3/30
135588/135588 [==============================] - 18s 132us/step - loss: 1348.4634 - val_loss: 1162.3615
Epoch 4/30
135588/135588 [==============================] - 17s 127us/step - loss: 1335.5915 - val_loss: 1160.5855
Epoch 5/30
135588/135588 [==============================] - 18s 134us/step - loss: 1324.9138 - val_loss: 1157.1709
Epoch 6/30
135588/135588 [==============================] - 18s 134us/step - loss: 1320.9364 - val_loss: 1151.4905
Epoch 7/30
135588/135588 [==============================] - 18s 130us/step - loss: 1311.3297 - val_loss: 1151.7594
Epoch 8/30
135588/135588 [==============================] - 19s 138us/step - loss: 1301.4330 - val_loss: 1148.3886
Epoch 9/30
135588/135588 [===

135588/135588 [==============================] - 19s 139us/step - loss: 1246.4067 - val_loss: 1146.0803
Epoch 11/30
135588/135588 [==============================] - 19s 141us/step - loss: 1244.1793 - val_loss: 1145.7803
Epoch 12/30
135588/135588 [==============================] - 19s 138us/step - loss: 1238.8154 - val_loss: 1142.4776
Epoch 13/30
135588/135588 [==============================] - 18s 135us/step - loss: 1233.1015 - val_loss: 1140.2750
Epoch 14/30
135588/135588 [==============================] - 19s 137us/step - loss: 1225.1271 - val_loss: 1145.3307
Epoch 15/30
135588/135588 [==============================] - 19s 138us/step - loss: 1226.5269 - val_loss: 1140.2690
Epoch 16/30
135588/135588 [==============================] - 18s 136us/step - loss: 1220.3472 - val_loss: 1139.6590
Epoch 17/30
135588/135588 [==============================] - 19s 139us/step - loss: 1219.2973 - val_loss: 1140.1490
Epoch 18/30
135588/135588 [==============================] - 19s 140us/step - loss: 

135588/135588 [==============================] - 18s 136us/step - loss: 1207.8705 - val_loss: 1139.8911
Epoch 21/30
135588/135588 [==============================] - 18s 136us/step - loss: 1200.6096 - val_loss: 1138.6058
Epoch 22/30
135588/135588 [==============================] - 18s 136us/step - loss: 1195.8688 - val_loss: 1139.5665
Epoch 23/30
135588/135588 [==============================] - 18s 131us/step - loss: 1196.3434 - val_loss: 1141.5087
Epoch 24/30
135588/135588 [==============================] - 19s 137us/step - loss: 1190.3771 - val_loss: 1138.4707
Epoch 25/30
135588/135588 [==============================] - 18s 136us/step - loss: 1190.0456 - val_loss: 1140.5115
Epoch 26/30
135588/135588 [==============================] - 18s 135us/step - loss: 1188.0599 - val_loss: 1137.1866
Epoch 27/30
135588/135588 [==============================] - 19s 137us/step - loss: 1182.8858 - val_loss: 1137.5942
Epoch 28/30
135588/135588 [==============================] - 19s 137us/step - loss: 

135588/135588 [==============================] - 18s 135us/step - loss: 1185.6241 - val_loss: 1138.3091
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 20s 148us/step - loss: 2116.1998 - val_loss: 1206.5501
Epoch 2/30
135588/135588 [==============================] - 19s 140us/step - loss: 1416.3263 - val_loss: 1188.1446
Epoch 3/30
135588/135588 [==============================] - 19s 144us/step - loss: 1391.3182 - val_loss: 1168.8870
Epoch 4/30
135588/135588 [==============================] - 20s 147us/step - loss: 1372.9346 - val_loss: 1163.3368
Epoch 5/30
135588/135588 [==============================] - 19s 141us/step - loss: 1362.2281 - val_loss: 1160.8660
Epoch 6/30
135588/135588 [==============================] - 19s 138us/step - loss: 1351.6266 - val_loss: 1155.5287
Epoch 7/30
135588/135588 [==============================] - 18s 133us/step - loss: 1340.0294 - val_loss: 1155.8911
Epoch 8/30
135588/135588 [==============

...

135588/135588 [==============================] - 28s 207us/step - loss: 1214.3793 - val_loss: 1137.7560
Epoch 25/30
135588/135588 [==============================] - 27s 201us/step - loss: 1211.4282 - val_loss: 1138.3809
Epoch 26/30
135588/135588 [==============================] - 28s 205us/step - loss: 1205.7220 - val_loss: 1137.4328
Epoch 27/30
135588/135588 [==============================] - 27s 202us/step - loss: 1198.5535 - val_loss: 1140.2237
Epoch 28/30
135588/135588 [==============================] - 29s 210us/step - loss: 1199.0898 - val_loss: 1139.3490
Epoch 29/30
135588/135588 [==============================] - 27s 201us/step - loss: 1195.3664 - val_loss: 1137.6377
Epoch 30/30
135588/135588 [==============================] - 27s 201us/step - loss: 1188.1626 - val_loss: 1138.4660
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 29s 216us/step - loss: 2110.6237 - val_loss: 1202.9859
Epoch 2/30
135588/135588 [========

135588/135588 [==============================] - 29s 210us/step - loss: 1340.6361 - val_loss: 1170.0717
Epoch 4/30
135588/135588 [==============================] - 28s 203us/step - loss: 1329.2369 - val_loss: 1164.9789
Epoch 5/30
135588/135588 [==============================] - 27s 202us/step - loss: 1319.7367 - val_loss: 1157.2249
Epoch 6/30
135588/135588 [==============================] - 28s 206us/step - loss: 1313.9019 - val_loss: 1158.8802
Epoch 7/30
135588/135588 [==============================] - 29s 211us/step - loss: 1306.4386 - val_loss: 1156.8475
Epoch 8/30
135588/135588 [==============================] - 28s 210us/step - loss: 1300.3337 - val_loss: 1150.6682
Epoch 9/30
135588/135588 [==============================] - 29s 210us/step - loss: 1293.7483 - val_loss: 1152.3449
Epoch 10/30
135588/135588 [==============================] - 28s 207us/step - loss: 1292.8610 - val_loss: 1148.3129
Epoch 11/30
135588/135588 [==============================] - 29s 212us/step - loss: 1284.3

135588/135588 [==============================] - 29s 210us/step - loss: 1280.3386 - val_loss: 1145.4167
Epoch 14/30
135588/135588 [==============================] - 29s 217us/step - loss: 1274.2175 - val_loss: 1144.0480
Epoch 15/30
135588/135588 [==============================] - 28s 206us/step - loss: 1266.6754 - val_loss: 1144.4092
Epoch 16/30
135588/135588 [==============================] - 29s 211us/step - loss: 1255.7963 - val_loss: 1145.6502
Epoch 17/30
135588/135588 [==============================] - 28s 209us/step - loss: 1256.7863 - val_loss: 1142.7322
Epoch 18/30
135588/135588 [==============================] - 29s 213us/step - loss: 1245.5386 - val_loss: 1141.9579
Epoch 19/30
135588/135588 [==============================] - 29s 217us/step - loss: 1240.0401 - val_loss: 1141.3946
Epoch 20/30
135588/135588 [==============================] - 29s 210us/step - loss: 1234.6653 - val_loss: 1141.5920
Epoch 21/30
135588/135588 [==============================] - 29s 211us/step - loss: 

135588/135588 [==============================] - 31s 225us/step - loss: 1203.6233 - val_loss: 1142.2702
Epoch 24/30
135588/135588 [==============================] - 29s 212us/step - loss: 1199.4590 - val_loss: 1140.0724
Epoch 25/30
135588/135588 [==============================] - 29s 214us/step - loss: 1198.5694 - val_loss: 1140.5301
Epoch 26/30
135588/135588 [==============================] - 30s 220us/step - loss: 1190.4251 - val_loss: 1139.9148
Epoch 27/30
135588/135588 [==============================] - 30s 219us/step - loss: 1191.6103 - val_loss: 1142.1868
Epoch 28/30
135588/135588 [==============================] - 29s 215us/step - loss: 1188.5767 - val_loss: 1142.0126
Epoch 29/30
135588/135588 [==============================] - 30s 219us/step - loss: 1187.0146 - val_loss: 1141.1724
Epoch 30/30
135588/135588 [==============================] - 29s 218us/step - loss: 1181.6518 - val_loss: 1139.8911
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [=======

135588/135588 [==============================] - 30s 222us/step - loss: 1394.5234 - val_loss: 1179.9220
Epoch 3/30
135588/135588 [==============================] - 30s 218us/step - loss: 1371.7671 - val_loss: 1176.8228
Epoch 4/30
135588/135588 [==============================] - 31s 228us/step - loss: 1359.1613 - val_loss: 1169.8034
Epoch 5/30
135588/135588 [==============================] - 32s 232us/step - loss: 1350.1370 - val_loss: 1159.8024
Epoch 6/30
135588/135588 [==============================] - 29s 216us/step - loss: 1340.2194 - val_loss: 1161.9867
Epoch 7/30
135588/135588 [==============================] - 30s 218us/step - loss: 1325.5196 - val_loss: 1154.4028
Epoch 8/30
135588/135588 [==============================] - 31s 232us/step - loss: 1320.4781 - val_loss: 1151.5985
Epoch 9/30
135588/135588 [==============================] - 31s 227us/step - loss: 1311.8714 - val_loss: 1151.7297
Epoch 10/30
135588/135588 [==============================] - 30s 218us/step - loss: 1303.07

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.

In [ ]:
'l3-nn': {
        'predictions': l2_predictions,
        'n_bags': 4,
        'model': Keras(nn_lr, lambda: {'l1': 1e-5, 'l2': 1e-5, 'n_epoch': 30, 'batch_size': 128, 'optimizer': SGD(3e-2, momentum=0.8, nesterov=True, decay=3e-5), 'callbacks': [ExponentialMovingAverage(save_mv_ave_model=False)]}),
    },

In [ ]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [ ]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adam','adadelta']),
         'wdecay':hp.choice('wdecay', np.linspace(0.00001,0.1,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], init='he_normal',
                        W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], init='he_normal',W_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.compile(loss='mae', optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.4210526315789474, 'hidden1_units': 455, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 272, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 69, 'optimizer': 'adadelta', 'wdecay': 0.032330000000000005}
Fold 0, MAE: 1163.148839039124
Fold 1, MAE: 1159.9899238609737
Fold 2, MAE: 1140.9771315408389
3-fold CV score: 1154.7052981469787
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.43333333333333335, 'hidden2_units': 134, 'hidden3_dropout': 0.10000000000000001, 'hidden3_units': 30, 'optimizer': 'adadelta', 'wdecay': 0.082830000000000001}
Fold 0, MAE: 1181.2090776051534
Fold 1, MAE: 1175.7242010975745
Fold 2, MAE: 1149.1617817128606
3-fold CV score: 1168.698353471863
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 113, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 22, 'optimizer': 'adam', '

Fold 0, MAE: 1160.1148852933434
Fold 1, MAE: 1153.5388360438872
Fold 2, MAE: 1146.284142179796
3-fold CV score: 1153.312621172342
Model Testing: {'hidden1_dropout': 0.44210526315789478, 'hidden1_units': 325, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 258, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 59, 'optimizer': 'adadelta', 'wdecay': 0.058590000000000003}
Fold 0, MAE: 1159.533359263806
Fold 1, MAE: 1154.5401700530679
Fold 2, MAE: 1152.34695090863
3-fold CV score: 1155.473493408501
Model Testing: {'hidden1_dropout': 0.43157894736842106, 'hidden1_units': 377, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 189, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 63, 'optimizer': 'adadelta', 'wdecay': 0.081820000000000004}
Fold 0, MAE: 1162.9307342647628
Fold 1, MAE: 1177.8290937843656
Fold 2, MAE: 1148.0584482485735
3-fold CV score: 1162.9394254325673
Model Testing: {'hidden1_dropout': 0.55789473684210522, 'hidden1_units': 455, 'hidden2_dropout

Fold 0, MAE: 1163.881212321844
Fold 1, MAE: 1164.1301011722203
Fold 2, MAE: 1150.2623383121809
3-fold CV score: 1159.4245506020818
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 532, 'hidden2_dropout': 0.40000000000000002, 'hidden2_units': 251, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adadelta', 'wdecay': 0.074749999999999997}
Fold 0, MAE: 1161.926003949864
Fold 1, MAE: 1171.028653969578
Fold 2, MAE: 1147.243582406554
3-fold CV score: 1160.0660801086653
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 300, 'hidden2_dropout': 0.33333333333333337, 'hidden2_units': 127, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adam', 'wdecay': 0.0020300000000000001}
Fold 0, MAE: 1157.4905136832626
Fold 1, MAE: 1157.7959576055189
Fold 2, MAE: 1145.165175664648
3-fold CV score: 1153.4838823178097
Model Testing: {'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.